In [1]:
import pandas as pd
import numpy as np
import time
import os
import glob
import json
from collections import defaultdict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression

## Configuration

In [2]:
# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../../results'
# ID of the training and test data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0005_19042021'
# Number of folds in cross-validation
CV = 6

print('Model trained with data: ' + DATA_ID)

Model trained with data: 0005_19042021


## Model

In [3]:
# Load data
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_{}.npy'.format(DATA_ID)))
X_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_test_{}.npy'.format(DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_{}.npy'.format(DATA_ID)))
Y_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_test_{}.npy'.format(DATA_ID)))

print('Train -> X: {}, Y: {}'.format(X_train.shape, Y_train.shape))
print('Test -> X: {}, Y: {}'.format(X_test.shape, Y_test.shape))

results_ls = []

Train -> X: (943372, 15), Y: (943372, 24)
Test -> X: (161200, 15), Y: (161200, 24)


In [15]:
params = {
    'alpha': 0.5
}

results = defaultdict(list)

tr_time = []
for target in range(Y_train.shape[1]):

    model = Lasso(**params, fit_intercept=True, random_state=0)
    
    t_start = time.time()
    model.fit(X_train, Y_train[:, target])
    tr_time.append(time.time() - t_start)
    
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    results['Train_MAE'].append(mean_absolute_error(Y_train[:, target], train_preds))
    results['Train_MSE'].append(mean_squared_error(Y_train[:, target], train_preds))
    results['Train_R2'].append(r2_score(Y_train[:, target], train_preds))
    results['Test_MAE'].append(mean_absolute_error(Y_test[:, target], test_preds))
    results['Test_MSE'].append(mean_squared_error(Y_test[:, target], test_preds))
    results['Test_R2'].append(r2_score(Y_test[:, target], test_preds))
    
print('Training time: {:.4f}'.format(sum(tr_time)))

results_summary = {}
results_summary['Training time'] = sum(tr_time)
# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results['_'.join([subset, loss])][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))
            results_summary[' '.join([subset, force, loss])] = '{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores))
            
results_ls.append(results_summary)

Training time: 11.4575
Train Fx MAE: 11.2775 ± 3.4866
Train Fx MSE: 259.7895 ± 163.8487
Train Fx R2: 0.1386 ± 0.0723
Train Fy MAE: 9.1404 ± 5.4528
Train Fy MSE: 249.1980 ± 279.3197
Train Fy R2: 0.0599 ± 0.0715
Train Fz MAE: 18.0691 ± 6.8765
Train Fz MSE: 663.8754 ± 614.7552
Train Fz R2: 0.1173 ± 0.0875
Test Fx MAE: 13.2300 ± 5.1843
Test Fx MSE: 353.5306 ± 287.0024
Test Fx R2: 0.0016 ± 0.0554
Test Fy MAE: 10.3156 ± 6.9224
Test Fy MSE: 340.7110 ± 422.1517
Test Fy R2: 0.0198 ± 0.0314
Test Fz MAE: 19.3499 ± 7.9991
Test Fz MSE: 823.6057 ± 904.4963
Test Fz R2: 0.0008 ± 0.1020


In [19]:
params = {
    'n_neighbors': 50
}

results = defaultdict(list)

tr_time = []
for target in range(Y_train.shape[1]):

    model = KNeighborsRegressor(**params, n_jobs=-1)
    
    t_start = time.time()
    model.fit(X_train, Y_train[:, target])
    tr_time.append(time.time() - t_start)
    
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    results['Train_MAE'].append(mean_absolute_error(Y_train[:, target], train_preds))
    results['Train_MSE'].append(mean_squared_error(Y_train[:, target], train_preds))
    results['Train_R2'].append(r2_score(Y_train[:, target], train_preds))
    results['Test_MAE'].append(mean_absolute_error(Y_test[:, target], test_preds))
    results['Test_MSE'].append(mean_squared_error(Y_test[:, target], test_preds))
    results['Test_R2'].append(r2_score(Y_test[:, target], test_preds))
    
print('Training time: {:.4f}'.format(sum(tr_time)))

results_summary = {}
results_summary['Training time'] = sum(tr_time)
# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results['_'.join([subset, loss])][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))
            results_summary[' '.join([subset, force, loss])] = '{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores))

Training time: 88.8137
Train Fx MAE: 2.8867 ± 0.7143
Train Fx MSE: 34.1048 ± 17.0101
Train Fx R2: 0.8816 ± 0.0231
Train Fy MAE: 2.6615 ± 1.3875
Train Fy MSE: 33.0138 ± 30.5818
Train Fy R2: 0.8418 ± 0.0488
Train Fz MAE: 4.1498 ± 1.4302
Train Fz MSE: 77.6918 ± 60.9860
Train Fz R2: 0.8917 ± 0.0147
Test Fx MAE: 12.4319 ± 4.1027
Test Fx MSE: 351.8722 ± 224.6564
Test Fx R2: -0.1222 ± 0.2788
Test Fy MAE: 11.1945 ± 6.9356
Test Fy MSE: 411.1543 ± 478.4239
Test Fy R2: -0.2516 ± 0.2415
Test Fz MAE: 18.3378 ± 8.5301
Test Fz MSE: 876.9917 ± 951.1303
Test Fz R2: -0.0563 ± 0.1415
